# Reformulated Code (error)

In [ ]:
#SAIDI calculation functions 

#first read the lookup table for identifying restore labels 
lookup = pd.read_parquet('outage_lookup.gz.parquet')
lookup['test_time'] = lookup['time'].apply(lambda x: int(x.replace(tzinfo=timezone.utc).timestamp()))

uplook = pd.DataFrame()
for i in lookup['core_id'].unique(): 
    relevant = lookup[lookup['core_id'] == i]
    relevant = relevant.sort_values('outage_time')
    relevant['trans_#'] = range(len(relevant))
    uplook = uplook.append(relevant)
lookup = uplook
lookup.head()


#ignoring restore clusters 
def ignore_restore(df):
    df['avg_out'] = df['outage_time'].apply(np.mean)
    df['avg_restore'] = df['restore_time'].apply(np.mean)
    df['std_out'] = df['outage_time'].apply(np.std)
    df['std_restore'] = df['restore_time'].apply(np.std)
    df['clust_saidi'] = (df['avg_restore'] - df['avg_out'])*df['cluster_size']
    no_rest_SAIDI = np.sum(df['clust_saidi'])/mean(df['sensors_reporting'])
    print(f'SAIDI when we ignore restore clustering: {no_rest_SAIDI}')
    return df


#this function appends the label of the restore cluster to the sensor in an outage cluster (does not deal with inverse noise)
def restore_clusters_exploded(db_out, db_restore, lookup, out_or_restore, other):   
    r_times = pd.DataFrame(db_restore.explode(other)[other])
    r_ids = pd.DataFrame(db_restore.explode('ids')['ids'])
    o_times = pd.DataFrame(db_out.explode(out_or_restore)[out_or_restore])
    o_ids = pd.DataFrame(db_out.explode('ids')['ids'])
    restore = r_times.combine_first(r_ids).reset_index()
    out = o_times.combine_first(o_ids).reset_index()
    restore_labels = []
    lat = []
    long = []
    multi = 0 
    for i in range(len(out)):
        time, eyed = out.iloc[i][out_or_restore], out.iloc[i]['ids']
        lookedup = lookup[(lookup[out_or_restore] == time) & (lookup['core_id'] == eyed) & (lookup['is_powered'] == False)]
        lat.append(lookedup['location_latitude'].values[0])
        long.append(lookedup['location_longitude'].values[0])
        if len(lookedup) == 1: 
            rest_time = lookedup[other].values[0]
            row = restore[(restore[other] == rest_time) & (restore['ids'] == eyed)]
            if len(row) == 1:
                restore_labels.append(row['labels'].values[0])
            else: 
                restore_labels.append(-1)
        elif len(lookedup) > 1: 
            multi += 1 
            rest_time = lookedup[other].values[0]
            row = restore[(restore[other] == rest_time) & (restore['ids'] == eyed)]
            if len(row) == 1:
                restore_labels.append(row['labels'].values[0])
            else: 
                restore_labels.append(-1)
        else: 
            print('problem: outage time not in the lookup table')
    out[other] = db_out.explode(other)[other].values
    out['restore_labels'] = restore_labels
    out['latitude'] = lat 
    out['longitude'] = long
    out['sensors_reporting'] = db_out.explode(out_or_restore)['sensors_reporting'].values
    return out


def db_w_labels(df, o_r_labels, other):
    percentage = []
    noise = []
    out_size = []
    for i in df[other].unique(): 
        a_df = df[df[other] == i]
        per = [len(a_df[o_r_labels].unique())]*len(a_df)
        n = [(len(a_df[a_df[o_r_labels] == -1]))/(len(a_df))]*len(a_df)
        o = [len(a_df)]*len(a_df)
        percentage.append(per)
        noise.append(n)
        out_size.append(o)
    percentage = [item for sublist in percentage for item in sublist]
    noise = [item for sublist in noise for item in sublist]
    out_size = [item for sublist in out_size for item in sublist]
    df['restore_groups'] = percentage 
    df['%_noise'] = noise
    df['out_size'] = out_size
    rest_count = pd.DataFrame(df.groupby(o_r_labels).count()[other])
    if other == 'labels':
        df = df.join(rest_count, on=o_r_labels, rsuffix='_').rename(columns={'labels_' : 'rest_size'})
        avg_restore_time = pd.DataFrame(df.groupby('restore_labels')['restore_time'].apply(np.mean))
        df = df.join(avg_restore_time, on='restore_labels', rsuffix='_').rename(columns={'restore_time_' : 'avg_rest_time'})
    if other == 'restore_labels':
        df = df.join(rest_count, on=other, rsuffix='_').rename(columns={'restore_labels_' : 'out_size'})
        avg_restore_time = pd.DataFrame(df.groupby('out_labels')['outage_time'].apply(np.mean))
        df = df.join(avg_restore_time, on='labels', rsuffix='_').rename(columns={'outage_time_' : 'avg_out_time'})
    return df


def avg_time_std_dur(df, out):
    mean_outage = out.groupby('labels')['outage_time'].apply(mean)
    mean_restore = out.groupby('labels')['restore_time'].apply(mean)
    std_outage = out.groupby('labels')['outage_time'].apply(np.std)
    std_restore = out.groupby('labels')['restore_time'].apply(np.std)
    num_sens = df.groupby('labels').count()['ids']
    
    df = df.join(mean_outage, on='labels', rsuffix='_').rename(columns={'outage_time_': 'mean_outage_time'})
    df = df.join(mean_restore, on='labels', rsuffix='_').rename(columns={'restore_time_': 'mean_restore_time'})
    df = df.join(std_outage, on='labels', rsuffix='__').rename(columns={'outage_time__': 'outage_time_stdev'})
    df = df.join(std_restore, on='labels', rsuffix='__').rename(columns={'restore_time__': 'restore_time_stdev'})
    df = df.join(num_sens, on='labels', rsuffix='__').rename(columns={'ids__': 'num_sens_out'})
    df['clust_saidi'] = (df['mean_restore_time'] - df['mean_outage_time'])*df['num_sens_out']
    return df 


def n_noise_saidi(df, out):
    no_noise_duration = avg_time_std_dur(df[df['restore_labels'] != -1], out)
    noise_duration = avg_time_std_dur(df[df['restore_labels'] == -1], out)
    no_noise_SAIDI = np.sum(no_noise_duration.groupby('labels').mean()['clust_saidi'])/ np.mean(no_noise_duration.groupby('labels').mean()['sensors_reporting'])
    noise_SAIDI = np.sum(noise_duration.groupby('labels').mean()['clust_saidi'])/ np.mean(noise_duration.groupby('labels').mean()['sensors_reporting'])
    print(f'SAIDI when we eliminate all noise: {noise_SAIDI}')
    print(f'SAIDI when we only include noise points: {no_noise_SAIDI}')
    return df


#noise clustered with nearest time 
def closest_time(df, avg_time, r_o_time):
    noise = df[df['restore_labels'] == -1]
    no_noise = df[df['restore_labels'] != -1]
    nearest = []
    for i in noise[r_o_time].values:
        nearest_label = pd.DataFrame(abs(avg_restore_time[r_o_time] - i)).sort_values(r_o_time).index[0]
        nearest.append(nearest_label)
    noise['nearest_time_labels'] = nearest
    noise = pd.DataFrame(noise['nearest_time_labels'])
    updated_df = df.join(noise)
    updated_df['restore_labels'] = updated_df['nearest_time_labels'].combine_first(df['restore_labels'])
    avg_restore_times = pd.DataFrame(updated_df.groupby('restore_labels')[r_o_time].apply(np.mean))
    if r_o_time == 'restore_time':
        closest_time_noise = updated_df.join(avg_time, on='restore_labels', rsuffix='_').rename(columns={'restore_time_' : 'avg_rest_times'})
    else: 
        closest_time_noise = updated_df.join(avg_time, on='restore_labels', rsuffix='_').rename(columns={'outage_time_' : 'avg_out_times'})        
    closest_time_noise = avg_time_std_dur_noise(closest_time_noise, 'time')
    return closest_time_noise


def avg_time_std_dur_noise(df, wrt):
    mean_outage = df.groupby('labels')['outage_time'].apply(mean)
    mean_restore = df.groupby('labels')['avg_rest_times'].apply(mean)
    std_outage = df.groupby('labels')['outage_time'].apply(np.std)
    std_restore = df.groupby('labels')['avg_rest_times'].apply(np.std)
    
    df = df.join(mean_outage, on='labels', rsuffix='_').rename(columns={'outage_time_': 'mean_outage_time'})
    df = df.join(mean_restore, on='labels', rsuffix='_').rename(columns={'avg_rest_times_': 'mean_restore_time'})
    df = df.join(std_outage, on='labels', rsuffix='__').rename(columns={'outage_time__': 'outage_time_stdev'})
    df = df.join(std_restore, on='labels', rsuffix='__').rename(columns={'avg_rest_times__': 'restore_time_stdev'})
    df['clust_saidi'] = (df['mean_restore_time'] - df['mean_outage_time'])*df['out_size']
    time_noise_SAIDI = np.sum(df.groupby('labels').mean()['clust_saidi'])/ np.mean(df.groupby('labels').mean()['sensors_reporting'])
    print(f'SAIDI when noise restores are clustered with the nearest restore with respect to {wrt}: {time_noise_SAIDI}')
    return df 


#noise clustered with nearest distance in space 

def closest_dist(df):
    noise = df[df['restore_labels'] == -1]
    no_noise = df[df['restore_labels'] != -1]
    nearest = []
    for n in range(len(noise)):
        lat = noise.iloc[n]['latitude']
        long = noise.iloc[n]['longitude']
        a_time = noise.iloc[n]['restore_time']
        calc_dist = pd.DataFrame(np.sqrt((no_noise['latitude'] - lat)**2 + (no_noise['longitude'] - long)**2)) 
        calc_dist['restore_labels'] = no_noise['restore_labels']
        nearest_labels = calc_dist[calc_dist[0] == calc_dist.sort_values(0).iloc[0][0]]['restore_labels'].values
        if len(nearest_labels) ==1:
            nearest_label = nearest_labels[0]
        else: 
            nearest_dist_times = avg_restore_time[avg_restore_time.index.isin(nearest_labels)]
            nearest_label = pd.DataFrame(abs(nearest_dist_times['restore_time'] - a_time)).sort_values('restore_time').index[0]
        nearest.append(nearest_label)
    noise['nearest_dist_labels'] = nearest
    noise = pd.DataFrame(noise['nearest_dist_labels'])
    updated_df = df.join(noise)
    updated_df['restore_labels'] = updated_df['nearest_dist_labels'].combine_first(df['restore_labels'])
    avg_restore_times = pd.DataFrame(updated_df.groupby('restore_labels')['restore_time'].apply(np.mean))
    updated_df = updated_df.join(avg_restore_times, on='restore_labels', rsuffix='_').rename(columns={'restore_time_' : 'avg_rest_times'})
    updated_df = avg_time_std_dur_noise(updated_df, 'space and time')
    return updated_df

In [ ]:
def cluster_db(time_label):
    data = two_days
    clustered_outages = run_dbscan(data, time_label)
    dbscan_formatted_clusters = prep_cluster_data(clustered_outages, time_label)
    dbscan_outages = run_convex_hull(dbscan_formatted_clusters, time_label)
    return dbscan_outages
    
db_out = cluster_db('outage_time')
db_restore = cluster_db('restore_time')
ignore_restore_clust = ignore_restore(db_out)
db_out_clusters = restore_clusters_exploded(db_out, db_restore, lookup, 'outage_time', 'restore_time')
db_out_clusters = db_w_labels(db_out_clusters, 'restore_labels', 'labels')
n_noise = n_noise_saidi(db_out_clusters, db_out_clusters)
avg_restore_time = pd.DataFrame(db_out_clusters[db_out_clusters['restore_labels'] != -1].groupby('restore_labels')['restore_time'].apply(np.mean))
db_nearest_time = closest_time(db_out_clusters, avg_restore_time, 'restore_time')
closest_dist_noise = closest_dist(db_out_clusters)
closest_dist_noise.head()

In [ ]:
#run SAIDI algorithm on restore clusters 

ignore_out_clust = ignore_restore(db_restore)
db_rest_clusters = restore_clusters_exploded(db_restore, db_out, lookup, 'restore_time', 'outage_time')
db_rest_clusters = n_noise_saidi(db_rest_clusters, db_rest_clusters)
avg_outage_time = pd.DataFrame(db_rest_clusters[db_rest_clusters['restore_labels'] != -1].groupby('restore_labels')['outage_time'].apply(np.mean))
db_nearest_time = closest_time(db_rest_clusters, avg_outage_time, 'outage_time')
# closest_dist_noise = closest_dist(db_rest_clusters)
# closest_dist_noise
#this gives an error for closest_time because when we call closest time on the restore clusters, we rename "avg_out_times"
#this throws and error when then we use avg_time_std_dur_noise, which requires the column to be called "avg_rest_times" 

# the old stuff

In [ ]:
#this are the OG functions closest_time and closest_dist 

# def closest_time(df):
#     noise = df[df['restore_labels'] == -1]
#     nearest = []
#     for i in noise['restore_time'].values:
#         nearest_label = pd.DataFrame(abs(avg_restore_time['restore_time'] - i)).sort_values('restore_time').index[0]
#         nearest.append(nearest_label)
#     noise['nearest_time_labels'] = nearest
#     noise = pd.DataFrame(noise['nearest_time_labels'])
#     updated_df = df.join(noise)
#     updated_df['restore_labels'] = updated_df['nearest_time_labels'].combine_first(df['restore_labels'])

#     return updated_df

# closest_time_noise = closest_time(db_out_clusters)
# avg_restore_times = pd.DataFrame(closest_time_noise.groupby('restore_labels')['restore_time'].apply(np.mean))
# closest_time_noise = closest_time_noise.join(avg_restore_times, on='restore_labels', rsuffix='_').rename(columns={'restore_time_' : 'avg_rest_times'})
# closest_time_noise

def avg_time_std_dur_noise(df):
    mean_outage = df.groupby('labels')['outage_time'].apply(mean)
    mean_restore = df.groupby('labels')['avg_rest_times'].apply(mean)
    std_outage = df.groupby('labels')['outage_time'].apply(np.std)
    std_restore = df.groupby('labels')['avg_rest_times'].apply(np.std)
    
    df = df.join(mean_outage, on='labels', rsuffix='_').rename(columns={'outage_time_': 'mean_outage_time'})
    df = df.join(mean_restore, on='labels', rsuffix='_').rename(columns={'avg_rest_times_': 'mean_restore_time'})
    df = df.join(std_outage, on='labels', rsuffix='__').rename(columns={'outage_time__': 'outage_time_stdev'})
    df = df.join(std_restore, on='labels', rsuffix='__').rename(columns={'avg_rest_times__': 'restore_time_stdev'})
    df['clust_saidi'] = (df['mean_restore_time'] - df['mean_outage_time'])*df['out_size']
    return df 

time_noise_duration = avg_time_std_dur_noise(closest_time_noise)
time_noise_SAIDI = np.sum(time_noise_duration.groupby('labels').mean()['clust_saidi'])/ np.mean(time_noise_duration.groupby('labels').mean()['sensors_reporting'])
time_noise_SAIDI

def closest_dist(df):
    noise = df[df['restore_labels'] == -1]
    nearest = []
    for n in range(len(noise)):
        lat = noise.iloc[n]['latitude']
        long = noise.iloc[n]['longitude']
        a_time = noise.iloc[n]['restore_time']
        calc_dist = pd.DataFrame(np.sqrt((df['latitude'] - lat)**2 + (df['longitude'] - long)**2)) 
        calc_dist['labels'] = db_out_clusters['labels']
        nearest_labels = calc_dist[calc_dist[0] == calc_dist.sort_values(0)[0].values]['labels']        
        nearest_dist_times = avg_restore_time[avg_restore_time.index.isin(nearest_labels)]
        nearest_label = pd.DataFrame(abs(nearest_dist_times['restore_time'] - a_time)).sort_values('restore_time').index[0]
        nearest.append(nearest_label)
    noise['nearest_dist_labels'] = nearest
    noise = pd.DataFrame(noise['nearest_dist_labels'])
    updated_df = df.join(noise)
    updated_df['restore_labels'] = updated_df['nearest_dist_labels'].combine_first(df['restore_labels'])

    return updated_df

closest_dist_noise = closest_dist(db_out_clusters)
avg_restore_times = pd.DataFrame(closest_dist_noise.groupby('restore_labels')['restore_time'].apply(np.mean))
closest_dist_noise = closest_dist_noise.join(avg_restore_times, on='restore_labels', rsuffix='_').rename(columns={'restore_time_' : 'avg_rest_times'})
closest_dist_noise



dist_noise_duration = avg_time_std_dur_noise(closest_dist_noise)
dist_noise_SAIDI = np.sum(dist_noise_duration.groupby('labels').mean()['clust_saidi'])/ np.mean(time_noise_duration.groupby('labels').mean()['sensors_reporting'])
dist_noise_SAIDI